In [ ]:
from config import Config
from pipeline import YargitayPipeline
import pandas as pd
import json
from processor import YargitaySemanticProcessor

def main():
    config = Config(
        CSV_FILE="/home/yapayzeka/ahsen_bulbul/data/cleaned10chunk.csv",
        TOKEN_SIZE=512,
        QDRANT_URL="http://localhost:6333",
        COLLECTION_NAME="try",
        EMBEDDING_DIM=512,
        BATCH_SIZE=100
    )
    pipeline = YargitayPipeline(config)

    while True:
        print("\n" + "="*60)
        print("🏛️ YARGITAY BGE-M3 SEMANTİK SİSTEM (Dense+Sparse)")
        print("="*60)
        print("1) Tam pipeline çalıştır (CSV -> chunks -> embed -> qdrant)")
        print("2) İnteraktif arama")
        print("3) Koleksiyon bilgilerini göster")
        print("4) Çıkış")
        choice = input("Seçiminiz (1-4): ").strip()
        if choice=="1":
            csv_path = input(f"CSV yolu (enter ile default: {config.CSV_FILE}): ").strip() or config.CSV_FILE
            ok = pipeline.full_pipeline(csv_path)
            print("✅ Tamamlandı" if ok else "❌ Hata çıktı")
        elif choice=="2":
            pipeline.interactive_search()
        elif choice=="3":
            info = pipeline.processor.get_collection_info()
            print(json.dumps(info, indent=2, ensure_ascii=False))
        elif choice=="4":
            print("👋 Görüşürüz")
            break
        else:
            print("❌ Geçersiz seçim")

if __name__ == "__main__":
    main()


/home/yapayzeka/ahsen_bulbul/ahsen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 192989.45it/s]


✅ Hazır - Cihaz: NVIDIA RTX A6000

1) Tam pipeline çalıştır
2) Arama
3) Çıkış
🗑️ Eski koleksiyon silindi: bge_hybrid_chunks
✅ Koleksiyon oluşturuldu: bge_hybrid_chunks (Dense+Sparse)


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{
  "collection_name": "bge_hybrid_chunks",
  "points_count": 59,
  "vectors_count": null,
  "status": "green"
}

1) Tam pipeline çalıştır
2) Arama
3) Çıkış
{
  "score": 0.69252634,
  "payload": null
}
{
  "score": 0.68916494,
  "payload": null
}
{
  "score": 0.65770495,
  "payload": null
}
{
  "score": 0.6522062,
  "payload": null
}
{
  "score": 0.65186155,
  "payload": null
}

1) Tam pipeline çalıştır
2) Arama
3) Çıkış
